In [1]:
# !sudo apt install swig

In [2]:
# !pip install quantstats
# !pip install scikit-learn
# !pip install torch
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

# Initializing

In [1]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
import torch

import numpy as np
import pandas as pd

from sklearn.preprocessing import MaxAbsScaler
from utils import PortfolioOptimizationEnv, custom_reward_function, sharpe_ratio_reward_function
from packages.custom_utils import PortfolioOptimizationEnv, custom_reward_function, sharpe_ratio_reward_function
from packages.models import DRLAgent
from packages.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Data Import and Preprocessing

In [2]:
# df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE) & (df_portfolio["date"] < "2020-12-31")]
# df_portfolio_test = df_portfolio[(df_portfolio["date"] >= "2021-01-01") & (df_portfolio["date"] < "2022-12-31")]


df_portfolio = pd.read_parquet('datasets/df_portfolio_train.parquet')
df_portfolio_test = pd.read_parquet('datasets/df_portfolio_test.parquet')

df_portfolio_train = df_portfolio[df_portfolio['date'] <= "2020-12-31"]
df_portfolio_vali = df_portfolio[df_portfolio['date'] >= "2021-01-01"]

# Feature Selection and DRL Model Parameters

In [3]:
import pandas as pd
from itertools import product
import numpy as np  # for NaN
import torch

FEATURE_NAMES = ["close",
               "vmd_mode_0",
                "vmd_mode_1",
                "vmd_mode_2",
                  #"high", 
                  #"low",
                'High_VIX',
                'Low_VIX',
                  #'Close_VIX',
                'High_SPY',
                'Low_SPY',
                'Close_SPY',
                  #'Volume_SPY',
                'Fear Greed', 
                'ratingScore',
                  #'ratingDetailsDCFScore',
                'ratingDetailsROEScore',
                  #'ratingDetailsROAScore',
                  #'ratingDetailsPEScore',
                  #'ratingDetailsPBScore',
                  #'expenses',
                  #'GDP', 
                'exports', 
                  #'imports',
                'pound_dollar_exchange_rate', 
                'unemployment',
                'ics',
                'TTM Revenue(Billion)',
                  #'TTM EBITDA(Billion)',
                  #'EBITDA Margin',
                'Price to Book Ratio',
                  #'PE Ratio',
                'Price to Sales Ratio',
                  #'Assets',
                  #'NetIncomeLoss',
                'Return on Equity(%)']

COMISSION_FEE = 0.001

# Define hyperparameter ranges
time_windows = [30, 60]
k_sizes = [3, 4]
conv_mids = [5, 20]
conv_finals = [5, 10, 20]
lr = [0.02]

# Function to calculate the final accumulative portfolio value
def calculate_cumulative_value(portfolio_values):
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    return final_value / initial_value

# Function to calculate the maximum drawdown
def calculate_max_drawdown(portfolio_values):
    portfolio_values = np.array(portfolio_values)
    running_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (portfolio_values - running_max) / running_max
    max_drawdown = drawdowns.min()
    
    return max_drawdown

# Function to calculate the Sharpe ratio
def calculate_sharpe_ratio(portfolio_values, risk_free_rate=0):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    excess_returns = returns - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    sharpe_ratio_annualized = sharpe_ratio * np.sqrt(252)
    
    return sharpe_ratio_annualized


# Define a function to calculate the metrics (this is just a placeholder; replace with actual functions)
def calculate_metrics(environment):
    final_portfolio_value = environment._asset_memory["final"][-1]
    max_drawdown = calculate_max_drawdown(environment._asset_memory["final"])
    sharpe_ratio = calculate_sharpe_ratio(environment._asset_memory["final"])
    cumulative_value = calculate_cumulative_value(environment._asset_memory["final"])
    return final_portfolio_value, max_drawdown, sharpe_ratio, cumulative_value

# DataFrame to store results
columns = ["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL","LEARNING_RATE", 
        'train_portfolio_value', 'train_drawdown', 'train_sharpe', 'train_cumulative',
        'test_portfolio_value', 'test_drawdown', 'test_sharpe', 'test_cumulative']

results_df = pd.DataFrame(columns=columns)

In [12]:
# Hyperparameter tuning loop with 5 episodes of training and error handling
for TIME_WINDOW, K_SIZE, CONV_MID, CONV_FINAL,LEARNING_RATE in list(product(time_windows, k_sizes, conv_mids, conv_finals, lr)):
    try:
        print(f"Training model with TIME_WINDOW={TIME_WINDOW}, K_SIZE={K_SIZE}, CONV_MID={CONV_MID}, CONV_FINAL={CONV_FINAL}, LEARNING_RATE={LEARNING_RATE}")
        # Update the policy_kwargs for the current set of hyperparameters
        policy_kwargs = {
        "initial_features": len(FEATURE_NAMES),
        "k_size": K_SIZE,
        "time_window": TIME_WINDOW,
        "conv_mid_features":CONV_MID,
        "conv_final_features":CONV_FINAL}

        # Create the environment for training and the test years
        
        environment = PortfolioOptimizationEnv(
            df_portfolio_train,
            initial_amount=100000,
            comission_fee_pct=COMISSION_FEE,
            time_window=TIME_WINDOW,
            features=FEATURE_NAMES,
            normalize_df=None,
            reward_function=custom_reward_function,
            reward_scaling=1.0
        )   

        vali_env = PortfolioOptimizationEnv(
            df_portfolio_vali,
            initial_amount=100000,
            comission_fee_pct=COMISSION_FEE,
            time_window=TIME_WINDOW,
            features=FEATURE_NAMES,
            normalize_df=None,
            reward_function=custom_reward_function,
            reward_scaling=1.0,
        )

        # Instantiate the model for each combination of hyperparameters
        # set PolicyGradient parameters
        model_kwargs = {
                "lr": LEARNING_RATE,
                "policy": EIIE,
                    "validation_env": vali_env,
                }

        EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)
        EIIE_policy = EIIE(time_window=TIME_WINDOW, device=device, initial_features=len(FEATURE_NAMES),
                            k_size=K_SIZE, conv_mid_features=CONV_MID, conv_final_features=CONV_FINAL)

        # Train the model for 10 episodes
        _, best_checkpoint = DRLAgent.train_model(EIIE_model, episodes=1)
        
        # Save the model policy after training
        model_filename = f"policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}_{LEARNING_RATE}.pt"
        torch.save(best_checkpoint, model_filename)
        
        EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
                            k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

        EIIE_policy.load_state_dict(torch.load(model_filename))

        environment.reset()
        DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy, learning_rate = LEARNING_RATE)
        train_final_value, train_drawdown, train_sharpe, train_cumulative = calculate_metrics(environment)

        print(f"Metrics - train_final_value: {train_final_value}, train_drawdown: {train_drawdown}, train_sharpe: {train_sharpe}, train_cumulative: {train_cumulative}")

        # Test evaluation
        vali_env.reset()
        DRLAgent.DRL_validation(EIIE_model, vali_env, policy=EIIE_policy, learning_rate = LEARNING_RATE)
        test_final_value, test_drawdown, test_sharpe, test_cumulative = calculate_metrics(vali_env)

        print(f"Metrics - test_final_value: {test_final_value}, test_drawdown: {test_drawdown}, test_sharpe: {test_sharpe}, test_cumulative: {test_cumulative}")


    except Exception as e:
        print(f"Error encountered with combination policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}: {e}")
        # If an error occurs, store NaN for all the metrics
        train_final_value = train_drawdown = train_sharpe = train_cumulative = np.nan
        test_final_value = test_drawdown = test_sharpe = test_cumulative = np.nan
 
    # Create a temporary DataFrame to store the current results
    temp_df = pd.DataFrame([{
        "TIME_WINDOW": TIME_WINDOW, "K_SIZE": K_SIZE, "CONV_MID": CONV_MID, "CONV_FINAL": CONV_FINAL, "LEARNING_RATE": LEARNING_RATE,
        "train_portfolio_value": train_final_value, "train_drawdown": train_drawdown, "train_sharpe": train_sharpe, "train_cumulative": train_cumulative,
        "test_portfolio_value": test_final_value, "test_drawdown": test_drawdown, "test_sharpe": test_sharpe, "test_cumulative": test_cumulative,
        }])
    
    # Use pd.concat instead of append to add the results to the main DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)
    
    try:
        results_df.to_csv('doublecheck.csv', index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
        
results_df

Training model with TIME_WINDOW=30, K_SIZE=3, CONV_MID=5, CONV_FINAL=5, LEARNING_RATE=0.02


  0%|          | 0/1 [00:00<?, ?it/s]

Initial portfolio value: 100000
Final portfolio value: 225506.734375
Final accumulative portfolio return: 2.25506734375
Maximum DrawDown: -0.3299484579219675
Sharpe ratio: 0.7678643964098895
Initial portfolio value: 100000
Final portfolio value: 127914.203125
Final accumulative portfolio return: 1.27914203125
Maximum DrawDown: -0.1482928230230205
Sharpe ratio: 0.889650661479365


100%|██████████| 1/1 [00:43<00:00, 43.72s/it]

Validation performance at episode 1: -2.4227851003360468
New best validation performance: -2.4227851003360468
Loaded best model based on validation performance.


Initial portfolio value: 100000
Final portfolio value: 236082.515625
Final accumulative portfolio return: 2.36082515625
Maximum DrawDown: -0.3442873783590086
Sharpe ratio: 0.7753553893536133
Metrics - train_final_value: 236082.515625, train_drawdown: -0.3442873783590086, train_sharpe: 0.7758049530879338, train_cumulative: 2.36082515625
Initial portfolio value: 100000
Final portfolio value: 128246.9921875
Final accumulative portfolio return: 1.282469921875
Maximum DrawDown: -0.1531440554980894
Sharpe ratio: 0.8779106998634418
Metrics - test_final_value: 128246.9921875, test_drawdown: -0.15314405549808943, test_sharpe: 0.8797721359949575, test_cumulative: 1.282469921875


C:\Users\dxcas\AppData\Local\Temp\ipykernel_23284\2066810198.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,30,3,5,5,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30,3,5,5,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30,3,5,5,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,3,5,5,0.02,236082.515625,-0.344287,0.775805,2.360825,128246.992188,-0.153144,0.879772,1.28247


In [33]:
results_df

,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,30,3,5,5,0.01,9.354249e+05,-0.384764,1.331227,9.354249,150983.859375,-0.288262,0.909782,1.509839
1,30,3,5,5,0.02,8.224098e+05,-0.362296,1.247078,8.224097,190940.890625,-0.210293,1.345313,1.909409
2,30,3,5,10,0.01,2.374833e+05,-0.342059,0.780448,2.374833,128788.664062,-0.153513,0.888851,1.287887
3,30,3,5,10,0.02,4.528810e+05,-0.284567,0.942631,4.528810,215230.218750,-0.187788,1.645605,2.152302
4,30,3,5,20,0.01,2.446373e+05,-0.352521,0.781757,2.446373,130033.421875,-0.159296,0.891152,1.300334
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,90,5,20,5,0.02,7.156793e+05,-0.280393,1.239699,7.156793,199785.375000,-0.217284,1.573390,1.997854
158,90,5,20,10,0.01,1.143282e+06,-0.314273,1.428601,11.432816,182986.265625,-0.205087,1.344036,1.829863
159,90,5,20,10,0.02,1.000000e+05,0.000000,NaN,1.000000,100000.000000,0.000000,NaN,1.000000
160,90,5,20,20,0.01,7.675396e+05,-0.385159,1.227981,7.675396,70939.335938,-0.386138,-0.527137,0.709393


In [ ]:
results_df2.to_csv('results_df2.csv', index=False)

In [35]:
results_df.to_csv('secondhyperparmatuning.csv', index=False)

In [36]:
FEATURE_NAMES = ['close',
                 'high',
                 'low',
                 'High_VIX',
                 'Low_VIX',
                 'Close_VIX',
                 'High_SPY',
                 'Low_SPY',
                 'Close_SPY',
                 'Volume_SPY',
                 'Fear Greed', 
                 'ratingScore',
                 'ratingDetailsDCFScore',
                 'ratingDetailsROEScore',
                 'ratingDetailsROAScore',
                 'ratingDetailsPEScore',
                 'ratingDetailsPBScore',
                 'expenses',
                 'GDP', 
                 'exports', 
                 'imports',
                 'pound_dollar_exchange_rate',
                 'unemployment',
                 'ics',
                 'TTM Revenue(Billion)',
                 'TTM EBITDA(Billion)',
                 'EBITDA Margin',
                 'Price to Book Ratio',
                 'PE Ratio',
                 'Price to Sales Ratio',
                 'Assets',
                 'NetIncomeLoss',
                 'Return on Equity(%)']

TIME_WINDOW = 50
COMISSION_FEE = 0.0025
K_SIZE = 4
CONV_MID = 5
CONV_FINAL= 20

environment = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        comission_fee_pct=COMISSION_FEE,
        time_window=TIME_WINDOW,
        features=FEATURE_NAMES,
        normalize_df=None,
        reward_function=custom_reward_function,
        reward_scaling=1.0
    )

# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01, 
    "policy": EIIE,
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "initial_features": len(FEATURE_NAMES),
    "k_size": K_SIZE,
    "time_window": TIME_WINDOW,
    "conv_mid_features":CONV_MID,
    "conv_final_features":CONV_FINAL}

EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

# DRL Model Training (Jan 2011- Dec 2020)

In [ ]:
DRLAgent.train_model(EIIE_model, episodes=5)
torch.save(EIIE_model.train_policy.state_dict(), "policy_EIIE(test).pt")

# DRL Model Evaluation (Jan 2021-Dec 2023)

In [ ]:
import os
import shutil

environment_2021 = PortfolioOptimizationEnv(
    df_portfolio_2021,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2022 = PortfolioOptimizationEnv(
    df_portfolio_2022,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2023 = PortfolioOptimizationEnv(
    df_portfolio_2023,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

EIIE_results = {
    "train": {},
    "2021": {},
    "2022": {},
    "2023": {}
}
# EI3_results = {
#     "train": {},
#     "2021": {},
#     "2022": {},
#     "2023": {}
# }

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.

EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
              k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

EIIE_policy.load_state_dict(torch.load("policy_EIIE(test).pt"))

environment.reset()
src_folder = "results/rl"
dst_folder = "results/train"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["train"]["value"] = environment._asset_memory["final"]
EIIE_results["train"]["actions"] = environment._actions_memory

src_folder = "results/rl"
dst_folder = "results/2021"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2021, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2021"]["value"] = environment_2021._asset_memory["final"]
EIIE_results["2021"]["actions"] = environment_2021._actions_memory

src_folder = "results/rl"
dst_folder = "results/2022"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2022, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2022"]["value"] = environment_2022._asset_memory["final"]
EIIE_results["2022"]["actions"] = environment_2022._actions_memory

src_folder = "results/rl"
dst_folder = "results/2023"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2023, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2023"]["value"] = environment_2023._asset_memory["final"]
EIIE_results["2023"]["actions"] = environment_2023._actions_memory


# EI3_policy = EI3(time_window=50, device=device)
# EI3_policy.load_state_dict(torch.load("policy_EI3.pt"))

# environment.reset()
# DRLAgent.DRL_validation(EI3_model, environment, policy=EI3_policy)
# EI3_results["train"]["value"] = environment._asset_memory["final"]
# environment_2021.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2021, policy=EI3_policy)
# EI3_results["2021"]["value"] = environment_2021._asset_memory["final"]
# environment_2022.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2022, policy=EI3_policy)
# EI3_results["2022"]["value"] = environment_2022._asset_memory["final"]
# environment_2023.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2023, policy=EI3_policy)
# EI3_results["2023"]["value"] = environment_2023._asset_memory["final"]

#uniform buy n hold
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
UBAH_results = {
    "train": {"value": [], "actions": []},
    "2021": {"value": [], "actions": []},
    "2022": {"value": [], "actions": []},
    "2023": {"value": [], "actions": []}
}

PORTFOLIO_SIZE = len(TEST_SET)

# train period
terminated = False
environment.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment.step(action)
    UBAH_results["train"]["actions"].append(action)
UBAH_results["train"]["value"] = environment._asset_memory["final"]

# 2021
terminated = False
environment_2021.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2021.step(action)
    UBAH_results["2021"]["actions"].append(action)
UBAH_results["2021"]["value"] = environment_2021._asset_memory["final"]

# 2022
terminated = False
environment_2022.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2022.step(action)
    UBAH_results["2022"]["actions"].append(action)
UBAH_results["2022"]["value"] = environment_2022._asset_memory["final"]

# 2023
terminated = False
environment_2023.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2023.step(action)
    UBAH_results["2023"]["actions"].append(action)
UBAH_results["2023"]["value"] = environment_2023._asset_memory["final"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["train"]["value"], label="EIIE")
#plt.plot(EI3_results["train"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

plt.plot(UBAH_results["2021"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2021"]["value"], label="EIIE")
#plt.plot(EI3_results["2021"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2021")
plt.legend()

plt.show()

plt.plot(UBAH_results["2022"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2022"]["value"], label="EIIE")
#plt.plot(EI3_results["2022"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2022")
plt.legend()

plt.show()

plt.plot(UBAH_results["2023"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2023"]["value"], label="EIIE")
#plt.plot(EI3_results["2023"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2023")
plt.legend()

plt.show()

# Portfolio Allocation by Trained DRL Model

In [ ]:
# Function to plot action memory
def plot_action_memory(actions, tickers, label):
    """Visualize portfolio allocation over time."""
    actions_df = pd.DataFrame(actions, columns=["cash"] + tickers)
    actions_df.plot(kind="line", figsize=(12, 8), title=f"Portfolio Allocation Breakdown - {label}")
    plt.ylabel("Portfolio Weight")
    plt.xlabel("Time Step")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot EIIE actions for the training period
plot_action_memory(EIIE_results["train"]["actions"], TEST_SET, "EIIE - Train")

# Plot EIIE actions for 2021
plot_action_memory(EIIE_results["2021"]["actions"], TEST_SET, "EIIE - 2021")

# Plot EIIE actions for 2022
plot_action_memory(EIIE_results["2022"]["actions"], TEST_SET, "EIIE - 2022")

# Plot EIIE actions for 2023
plot_action_memory(EIIE_results["2023"]["actions"], TEST_SET, "EIIE - 2023")

# Similarly, plot UBAH actions for each period
# plot_action_memory(UBAH_results["train"]["actions"], TEST_SET, "UBAH - Train")
# plot_action_memory(UBAH_results["2021"]["actions"], TEST_SET, "UBAH - 2021")
# plot_action_memory(UBAH_results["2022"]["actions"], TEST_SET, "UBAH - 2022")
# plot_action_memory(UBAH_results["2023"]["actions"], TEST_SET, "UBAH - 2023")

In [ ]:
results_df2 = results_df2[["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL", 
            'train_portfolio_value', 'train_drawdown', 'train_sharpe','train_cumulative',
            '2021_portfolio_value', '2021_drawdown', '2021_sharpe','2021_cumulative',
             '2022_portfolio_value', '2022_drawdown', '2022_sharpe','2022_cumulative',
             '2023_portfolio_value', '2023_drawdown', '2023_sharpe','2023_cumulative']]